In [0]:
import pickle
import boto3
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from nameparser import HumanName

In [0]:
from pyspark.sql import SparkSession
sc = spark.sparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField
sqlContext = SQLContext(sc)

/databricks/spark/python/pyspark/sql/context.py:117: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
base_save_path = "s3://author-disambiguation/V1/redshift_temp_dir/"
iteration_save_path = "s3://author-disambiguation/V1/"

### Getting all data

In [0]:
query = \
"""
select count(*)
from mid.author
"""

In [0]:
all_data = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
all_data.show()

+---------+
 count|
+---------+
251232707|
+---------+

### Save Static Tables

In [0]:
query = \
"""
select author_id, normalized_name, display_name, paper_count,last_known_affiliation_id
from mid.author
"""

In [0]:
authors = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
authors \
.write.mode('overwrite').parquet(f"{iteration_save_path}static_authors")

In [0]:
query = \
"""
select paper_id, original_title, TO_DATE(publication_date,'YYYY-MM-DD') as publication_date, journal_id
from mid.work
"""

In [0]:
works = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
works \
.write.mode('overwrite').parquet(f"{iteration_save_path}static_works")

In [0]:
query = \
"""
select distinct journal_id, display_name as journal
from mid.journal
"""

In [0]:
journals = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
journals \
.write.mode('overwrite').parquet(f"{iteration_save_path}static_journals")

In [0]:
query = \
"""
select paper_id, author_id, original_author, author_sequence_number, cast(original_affiliation as TEXT), original_orcid
from mid.affiliation
"""

In [0]:
affiliations = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
affiliations \
.write.mode('overwrite').parquet(f"{iteration_save_path}static_affiliations")

In [0]:
affiliations.dropDuplicates(subset=['paper_id', 'original_author', 'author_sequence_number']).groupBy('paper_id').count().filter(F.col('count')<300).count()

Out[78]: 220474025

In [0]:
query = \
"""
select distinct paper_id, paper_reference_id
from mid.citation
"""

In [0]:
citations = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
citations \
.write.mode('overwrite').parquet(f"{iteration_save_path}static_citations")

In [0]:
citations.groupBy('paper_id').count().filter(F.col('count')<300).count()

Out[62]: 69308347

In [0]:
69308347/69395346

Out[63]: 0.9987463280318539

In [0]:
query = \
"""
select affiliation_id, display_name, normalized_name, mag_normalized_name, ror_id, city, region, country
from mid.institution
"""

In [0]:
institutions = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
institutions \
.write.mode('overwrite').parquet(f"{iteration_save_path}static_institutions")

### Get data for author disambiguation

In [0]:
get_papers = \
f"""select paper_id, original_title, EXTRACT(YEAR from TO_DATE(publication_date,'YYYY-MM-DD')) as pub_year, 
   journal_id
   from mid.work"""

get_journal_name = \
f"""select distinct journal_id, display_name as journal
    from mid.journal"""

attach_authors = \
f"""select paper_id, author_id as mag_author_id, original_author as author_name, 
           concat(cast(paper_id as VARCHAR(15)), concat('_', cast(author_sequence_number as VARCHAR(5)))) as paper_author_id,  
           author_sequence_number as seq_no, listagg(cast(original_affiliation as TEXT),'::::') as affiliation
    from mid.affiliation
    where original_author is not null
    group by paper_id, author_id, original_author, author_sequence_number"""

attach_coauthors = \
f"""select paper_id, listagg(original_author,'::::') as coauthors
    from (select distinct paper_id, original_author, author_sequence_number, ROW_NUMBER() OVER (PARTITION BY paper_id) as row_number
          from mid.affiliation) as all_authors
    where row_number <= 300
    group by paper_id"""

attach_references = \
f"""select paper_id, listagg(cast(paper_reference_id as VARCHAR(16)),'::::') as references
    from (select distinct paper_id, paper_reference_id, ROW_NUMBER() OVER (PARTITION BY paper_id) as row_number
          from mid.citation) as all_refs
    where row_number <= 300
    group by paper_id"""

In [0]:
testing_df = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", attach_references) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
full_query = \
f"""
select a.paper_id as UID, a.original_title title, a.pub_year, e.journal, b.mag_author_id, 
       b.author_name as author, b.paper_author_id, b.affiliation, b.seq_no, c.coauthors, d.references
from ({get_papers}) a
left join ({attach_authors}) b
on a.paper_id=b.paper_id
left join ({attach_coauthors}) c
on a.paper_id=c.paper_id
left join ({attach_references}) d
on a.paper_id=d.paper_id
left join({get_journal_name}) e
on a.journal_id=e.journal_id
"""

In [0]:
raw_input_data = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", full_query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
raw_input_data \
.write.mode('overwrite').parquet(f"{iteration_save_path}raw_author_disam_input_data")

### Reading the Saved Tables

In [0]:
# authors = spark.read.parquet(f"{iteration_save_path}static_authors")
# works = spark.read.parquet(f"{iteration_save_path}static_works")
# journals = spark.read.parquet(f"{iteration_save_path}static_journals")
affiliations = spark.read.parquet(f"{iteration_save_path}static_affiliations")
# institutions = spark.read.parquet(f"{iteration_save_path}static_institutions")
# citations = spark.read.parquet(f"{iteration_save_path}static_citations")
# raw_input_data = spark.read.parquet(f"{iteration_save_path}raw_author_disam_input_data").filter(F.col('author')!='')

In [0]:
works.printSchema()

root
 |-- paper_id: long (nullable = true)
 |-- original_title: string (nullable = true)
 |-- publication_date: date (nullable = true)
 |-- journal_id: long (nullable = true)



In [0]:
affiliations.cache().count()

Out[45]: 614116887

In [0]:
affiliations.filter(F.col('original_orcid')!='').select(F.min(F.col('paper_id')), F.max(F.col('paper_id'))).show()

+-------------+-------------+
|min(paper_id)|max(paper_id)|
+-------------+-------------+
|   2908575976|   4296761487|
+-------------+-------------+



In [0]:
affiliations.filter(F.col('original_orcid')=='0000-0001-9695-4543') \
.select('original_orcid','original_author').show(25, truncate=False)

+-------------------+---------------+
|original_orcid     |original_author|
+-------------------+---------------+
|0000-0001-9695-4543|Valeriy Lakhno |
|0000-0001-9695-4543|Valerii Lakhno |
|0000-0001-9695-4543|Valery Lakhno  |
|0000-0001-9695-4543|V. Lakhno      |
|0000-0001-9695-4543|V. Lakhno      |
|0000-0001-9695-4543|V. A. Lakhno   |
|0000-0001-9695-4543|Valerii Lakhno |
|0000-0001-9695-4543|Valery Lakhno  |
|0000-0001-9695-4543|Lakhno Valeriy |
+-------------------+---------------+



In [0]:
affiliations.filter(F.col('original_orcid')!='') \
.groupby('original_orcid') \
.agg(F.collect_set(F.col('original_author')).alias('author_names')) \
.withColumn('author_names_len', F.size(F.col('author_names'))) \
.filter(F.col('author_names_len') > 5) \
.show(25, truncate=False)

+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+
|original_orcid     |author_names                                                                                                                                                                                           |author_names_len|
+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+
|0000-0001-6206-2133|[S. A. Melchenko, Yu.M. Zabrodskaya, Yu. M. Zabrodskaya, J. M. Zabrodskaya, Yu M Zabrodskaya, Yu. M. Zabrodskaia]                                                                                      |6               |
|0000-0001-6268-6552|[José Padrón, José M. P

In [0]:
works.show(5)

+--------+--------------------+----------------+----------+
|paper_id|      original_title|publication_date|journal_id|
+--------+--------------------+----------------+----------+
|     236|CHAPTER 11 – Stat...|      2001-01-01|      null|
|    1280|University of Mic...|      1955-08-01|      null|
|    1638|Benvenuto Cellini...|      1834-01-01|      null|
|    1682|Capítulo 15 – Apl...|      2005-01-01|      null|
|    3190|Beiträge zur Frag...|      1933-01-01|2752137261|
+--------+--------------------+----------------+----------+
only showing top 5 rows



In [0]:
works.select(F.max(F.col('paper_id'))).show()

+-------------+
|max(paper_id)|
+-------------+
|   4291252484|
+-------------+



In [0]:
raw_input_data.cache().count()

Out[5]: 604871070

#### Quick Data Quality Checks

In [0]:
raw_input_data.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in raw_input_data.columns]).show()

+---+-----+--------+-------+-------------+------+---------------+-----------+------+---------+----------+
uid|title|pub_year|journal|mag_author_id|author|paper_author_id|affiliation|seq_no|coauthors|references|
+---+-----+--------+-------+-------------+------+---------------+-----------+------+---------+----------+
 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+-----+--------+-------+-------------+------+---------------+-----------+------+---------+----------+

In [0]:
raw_input_data \
.withColumn("affiliation", turn_string_into_list_udf(F.col('affiliation'))) \
.withColumn("coauthors", turn_string_into_list_udf(F.col('coauthors'))) \
.withColumn("references", turn_string_into_list_udf(F.col('references'))) \
.withColumn("coauthors", transform_coauthors_udf(F.col('coauthors'))) \
.withColumn("coauthor_len", F.size(F.col('coauthors'))) \
.withColumn("affiliation_len", F.size(F.col('affiliation'))) \
.withColumn("references_len", F.size(F.col('references'))) \
.select("pub_year", "coauthor_len", "affiliation_len", "references_len") \
.groupby('pub_year').mean().orderBy('pub_year') \
.filter(F.col('pub_year')>1975).show(50)

+--------+-------------+------------------+--------------------+-------------------+
pub_year|avg(pub_year)| avg(coauthor_len)|avg(affiliation_len)|avg(references_len)|
+--------+-------------+------------------+--------------------+-------------------+
 1976| 1976.0|3.5444656326763004| 1.005909737434547| 3.6560007410066597|
 1977| 1977.0|3.9397931152698926| 1.005699374218141| 3.8671707891845486|
 1978| 1978.0|3.7780280709894445| 1.0060958125507482| 4.017972392993852|
 1979| 1979.0| 3.67181633117668| 1.0060754122601177| 4.2205211283172686|
 1980| 1980.0|3.6698145629462857| 1.0058025774239254| 4.0952232706003135|
 1981| 1981.0| 4.094913386168779| 1.0062608296374893| 4.452053307024198|
 1982| 1982.0|4.1526760372822435| 1.0060388879555278| 4.486606432658984|
 1983| 1983.0| 4.228749423100933| 1.0064104182711124| 4.60514117018277|
 1984| 1984.0| 4.31914925629431| 1.0063378934199014| 4.769900484901016|
 1985| 1985.0| 4.465709670895231| 1.0061478994029804| 5.040170273419818|
 1986| 1986.0| 4.36273025012484| 1.0060787556822577| 5.188195556409962|
 1987| 1987.0| 4.625152845501498| 1.0059118832604177| 5.252142962556895|
 1988| 1988.0|4.6349275760499395| 1.0053448683462294| 5.534922847887342|
 1989| 1989.0| 4.967871391686527| 1.0053606553218586| 5.652761963149169|
 1990| 1990.0| 6.596077780210638| 1.0048267418836274| 5.804147758826801|
 1991| 1991.0| 6.505672969354904| 1.0052083740942612| 6.0739648768944186|
 1992| 1992.0| 7.325441568313953| 1.005515239010127| 6.448401832549511|
 1993| 1993.0| 7.442716646674284| 1.0052643125991902| 6.322133609902545|
 1994| 1994.0| 7.484321782346917| 1.005325912866308| 6.743051921148869|
 1995| 1995.0| 9.08788372152299| 1.0054405580961745| 6.994167117991875|
 1996| 1996.0| 8.656917728490404| 1.0062058118597166| 6.943265770351689|
 1997| 1997.0| 8.307136037574981| 1.0059577538982785| 7.337056031301683|
 1998| 1998.0| 8.206653436794538| 1.0060946963116852| 7.68828561420503|
 1999| 1999.0| 8.16551558940821| 1.0062300196458525| 8.970450288160926|
 2000| 2000.0| 7.911615579016571| 1.0060926821342775| 8.833258018447108|
 2001| 2001.0| 7.853645879079273| 1.0062855269826223| 9.107932274786007|
 2002| 2002.0| 7.765062849243263| 1.006390413566215| 9.430672873078992|
 2003| 2003.0| 6.857638560733638| 1.00667576013652| 9.511639401916446|
 2004| 2004.0| 8.517404374994596| 1.0072741165246828| 9.925740393352866|
 2005| 2005.0| 8.567483873431783| 1.0077946074317468| 10.49533437762454|
 2006| 2006.0| 8.59005709264874| 1.0084586759324325| 10.731835760934255|
 2007| 2007.0| 9.283014515979042| 1.0088324910221056| 11.13996023049373|
 2008| 2008.0| 9.38310606694475| 1.0091660956970443| 11.241211387874536|
 2009| 2009.0|15.375789890090788| 1.0098262592982366| 11.293597547838763|
 2010| 2010.0|13.266998480411816| 1.0105614863758323| 11.780234409366805|
 2011| 2011.0|13.030094615607084| 1.011858468363002| 12.7441165904193|
 2012| 2012.0| 18.8202134084449| 1.0125711160961055| 13.43897724657726|
 2013| 2013.0|15.714697063685895| 1.013615484992953| 14.06751626831831|
 2014| 2014.0|14.519966812040876| 1.013547791821421| 14.793212712334117|
 2015| 2015.0|14.659715566980585| 1.0150501314222433| 15.439320073120523|
 2016| 2016.0|19.079164764986995| 1.0158877767354824| 14.860523517563562|
 2017| 2017.0|15.351116863040163| 1.0184894168278418| 16.040409643293767|
 2018| 2018.0| 16.53857226628665| 1.0211546742880182| 16.504875933534812|
 2019| 2019.0|15.667150040480987| 1.0227751642025258| 17.87675801263855|
 2020| 2020.0|14.135846486226137| 1.0254717750558155| 18.3937380137707|
 2021| 2021.0|18.367056783461347| 1.0285841904319675| 21.694587787768484|
 2022| 2022.0|25.995612611027106| 1.0721302957490342| 25.35498079061955|
 2023| 2023.0|10.979903705254344| 1.0384132300607076| 25.494138580699182|
 2024| 2024.0| 7.958348968105065| 1.0138836772983115| 20.18686679174484|
 2025| 2025.0| 7.71875| 1.00625| 1.19375|
+--------+-------------+------------------+--------------------+-------------------+
only showing top 50 rows

In [0]:
raw_input_data \
.select('pub_year') \
.groupby('pub_year').count() \
.orderBy('pub_year') \
.filter(F.col('pub_year')>1975).show(50)

+--------+--------+
pub_year| count|
+--------+--------+
 1976| 1910914|
 1977| 2036013|
 1978| 2155250|
 1979| 2270957|
 1980| 2534391|
 1981| 2578572|
 1982| 2690396|
 1983| 2851452|
 1984| 3057325|
 1985| 3218823|
 1986| 3412376|
 1987| 3679369|
 1988| 3912725|
 1989| 4218141|
 1990| 4649306|
 1991| 4727771|
 1992| 4968597|
 1993| 5264505|
 1994| 5575570|
 1995| 6002693|
 1996| 6522112|
 1997| 6911833|
 1998| 7362631|
 1999| 7660650|
 2000| 8499869|
 2001| 8960267|
 2002| 9711891|
 2003|10856142|
 2004|11914849|
 2005|12986799|
 2006|14426726|
 2007|15560163|
 2008|17090810|
 2009|19007233|
 2010|20229823|
 2011|21527738|
 2012|23605064|
 2013|24978765|
 2014|25769587|
 2015|26810530|
 2016|28899890|
 2017|27947339|
 2018|28541919|
 2019|29996798|
 2020|32430445|
 2021|32812054|
 2022|16705380|
 2023| 19108|
 2024| 2665|
 2025| 160|
+--------+--------+
only showing top 50 rows

#### Get data into final format for feeding to blocking function

In [0]:
raw_input_data.sample(0.00001).select('uid','author') \
.withColumn('author_split', check_author_name_udf(F.col('author'))) \
.withColumn('author_last_name_len', last_name_check_udf(F.col('author'))) \
.filter(F.col('author_last_name_len')==2) \
.show(100, truncate=False)

+----------+-------------+-------------+--------------------+
uid |author |author_split |author_last_name_len|
+----------+-------------+-------------+--------------------+
4282023997|xiangping qu |xiangping qu |2 |
365248090 |雅明 石塚 |雅明 石塚 |2 |
653842788 |逹 山本 |逹 山本 |2 |
304198829 |良之 帶刀 |良之 帶刀 |2 |
4210363173|Yangping Li |Yangping Li |2 |
2392957748|Cao Ji |Cao Ji |2 |
2508830026|竹内 昭彦 |竹内 昭彦 |2 |
2042087331|Kerth Wj |WJ Kerth |2 |
16845551 |誠 清水 |誠 清水 |2 |
585487600 |武志 古川 |武志 古川 |2 |
2074842342|X. Ma |X. Ma |2 |
1538704079|誠 飯田 |誠 飯田 |2 |
356789053 |昭子 中村 |昭子 中村 |2 |
401867376 |孝 岩下 |孝 岩下 |2 |
2943952301|Ziao Fu |Ziao Fu |2 |
379237897 |隆 横山 |隆 横山 |2 |
2410207359|Mullany Cj |CJ Mullany |2 |
2783443174|Qingxu Yu |Qingxu Yu |2 |
3183337926|淳太郎 芦刈 |淳太郎 芦刈 |2 |
2076442294|Yunkai Li |Yunkai Li |2 |
3163376438|Lin Ye |Lin Ye |2 |
650315780 |一成 金子 |一成 金子 |2 |
2951651696|Gui Lu |Gui Lu |2 |
2915523118|健 金子 |健 金子 |2 |
2596642980|G. Xu |G. Xu |2 |
2913220901|昭彦 松峯 |昭彦 松峯 |2 |
602842247 |眞一 高田 |眞一 高田 |2 |
2904807669|Rongqing Xu |Rongqing Xu |2 |
407189219 |一寿 藤本 |一寿 藤本 |2 |
2981731292|Jiyuan Tu |Jiyuan Tu |2 |
640450218 |克之 吉田 |克之 吉田 |2 |
316858981 |隆一 青木 |隆一 青木 |2 |
2534511095|Erlong Li |Erlong Li |2 |
2747335770|直章 石田 |直章 石田 |2 |
2328453635|Xiaodong Xu |Xiaodong Xu |2 |
3089725974|Long Qu |Long Qu |2 |
2404829041|Xiao-Jiang Yu|Xiao-Jiang Yu|2 |
948789189 |貴子 上川 |貴子 上川 |2 |
2273662850|浩之 岩瀬 |浩之 岩瀬 |2 |
386088018 |武彦 伊東 |武彦 伊東 |2 |
1577259104|幸泰 小澤 |幸泰 小澤 |2 |
647455250 |徹 吉田 |徹 吉田 |2 |
2555024076|Wenqing Li |Wenqing Li |2 |
2135715702|Y. Li |Y. Li |2 |
1003397019|Lloyd Bj |BJ Lloyd |2 |
2047103902|Jiann-Ming Wu|Jiann-Ming Wu|2 |
4225255716|Yanhong Li |Yanhong Li |2 |
615049692 |靖子 古賀 |靖子 古賀 |2 |
4285612929|Zhuo Li |Zhuo Li |2 |
339284904 |慶一 北條 |慶一 北條 |2 |
2949379042|Youqiao Ma |Youqiao Ma |2 |
2394524447|Burns Rp |RP Burns |2 |
364968913 |康雄 大西 |康雄 大西 |2 |
2736322659|Sheng-mei Ma |Sheng-mei Ma |2 |
653599318 |哲也 杉正 |哲也 杉正 |2 |
2921462187|Bosch Fx |FX Bosch |2 |
2991426221|Yan Li |Yan Li |2 |
2099199527|Maohua Lu |Maohua Lu |2 |
4256292937|Hui Li |Hui Li |2 |
658806768 |雅英 前田 |雅英 前田 |2 |
2399978101|Goel Ka |Goel Ka |2 |
378823269 |照弘 瀬島 |照弘 瀬島 |2 |
306678905 |俊毅 高橋 |俊毅 高橋 |2 |
3091265663|Ping Lu |Ping Lu |2 |
2398837232|智宏 山倉 |智宏 山倉 |2 |
586975897 |清志 川村 |清志 川村 |2 |
617356572 |俊男 山名 |俊男 山名 |2 |
2548625455|Jun Hu |Jun Hu |2 |
2900019138|Yunfeng Ge |Yunfeng Ge |2 |
2414005015|Simonov Pv |PV Simonov |2 |
608892426 |幸嗣 武川 |幸嗣 武川 |2 |
2574501932|亙 小沢 |亙 小沢 |2 |
2187074594|Xiameng Xu |Xiameng Xu |2 |
1580128959|泉 山口 |泉 山口 |2 |
2765904729|紘司 市川 |紘司 市川 |2 |
2346752951|誠吾 佐藤 |誠吾 佐藤 |2 |
11516439 |努 田川 |努 田川 |2 |
2503913353|佳菜子 内田 |佳菜子 内田 |2 |
592060189 |玉堂 川合 |玉堂 川合 |2 |
2564566551|Xuming He |Xuming He |2 |
2785903318|Ran Li |Ran Li |2 |
596857085 |慶徳 平井 |慶徳 平井 |2 |
433858459 |良平 田崎 |良平 田崎 |2 |
360034949 |達生 鳥居 |達生 鳥居 |2 |
75320826 |Kempf Hg |HG Kempf |2 |
2413081325|Chaturvedi Vk|VK Chaturvedi|2 |
2275081203|Ogola Pe |Ogola Pe |2 |
2463966808|Qiang Wu |Qiang Wu |2 |
4239939555|Bohai Ji |Bohai Ji |2 |
3175686446|昭美 青木 |昭美 青木 |2 |
618169127 |秀之 高津 |秀之 高津 |2 |
120594693 |清一 鷲田 |清一 鷲田 |2 |
589879033 |隆 栗原 |隆 栗原 |2 |
2159929259|Sen Hu |Sen Hu |2 |
4210580258|Ing-Song Yu |Ing-Song Yu |2 |
3107855858|Lin Yu |Lin Yu |2 |
4229587438|Suxia Li |Suxia Li |2 |
1430482963|中山 耕造 |中山 耕造 |2 |
426550260 |賢次 三枝 |賢次 三枝 |2 |
2512465038|宮本 清茂 |宮本 清茂 |2 |
+----------+-------------+-------------+--------------------+
only showing top 100 rows

In [0]:
raw_input_data.filter(F.col('uid')==4200068434).show(truncate=False)

+----------+------------------------------------------+--------+------------------------------------------------------+-------------+---------------+---------------+-----------+------+------------------------------------------------+----------+
uid |title |pub_year|journal |mag_author_id|author |paper_author_id|affiliation|seq_no|coauthors |references|
+----------+------------------------------------------+--------+------------------------------------------------------+-------------+---------------+---------------+-----------+------+------------------------------------------------+----------+
4200068434|Shareholders Lock-Up in Close Corporations|2021 |Vestnik èkonomičeskogo pravosudiâ Rossijskoj Federacii|4201926957 |D.S. Pavlova |4200068434_3 | |3 |D.S. Pavlova::::O.A. Ryabtsev::::Yu.Yu. Zakaznov|null |
4200068434|Shareholders Lock-Up in Close Corporations|2021 |Vestnik èkonomičeskogo pravosudiâ Rossijskoj Federacii|4200932120 |Yu.Yu. Zakaznov|4200068434_1 | |1 |D.S. Pavlova::::O.A. Ryabtsev::::Yu.Yu. Zakaznov|null |
4200068434|Shareholders Lock-Up in Close Corporations|2021 |Vestnik èkonomičeskogo pravosudiâ Rossijskoj Federacii|4201926958 |O.A. Ryabtsev |4200068434_4 | |4 |D.S. Pavlova::::O.A. Ryabtsev::::Yu.Yu. Zakaznov|null |
+----------+------------------------------------------+--------+------------------------------------------------------+-------------+---------------+---------------+-----------+------+------------------------------------------------+----------+

In [0]:
raw_input_data.printSchema()

root
-- uid: long (nullable = true)
-- title: string (nullable = true)
-- pub_year: integer (nullable = true)
-- journal: string (nullable = true)
-- mag_author_id: long (nullable = true)
-- author: string (nullable = true)
-- paper_author_id: string (nullable = true)
-- affiliation: string (nullable = true)
-- seq_no: integer (nullable = true)
-- coauthors: string (nullable = true)
-- references: string (nullable = true)

In [0]:
def turn_string_into_list(col_string):
    if isinstance(col_string, str):
        return col_string.split("::::")
    else:
        return []
    
def add_W_to_works_and_authors(work_author):
    if isinstance(work_author, str):
        return f"W{work_author}"
    else:
        return f"W{str(work_author)}"
      
def add_W_to_references(references):
    return [f"W{str(x)}" for x in references]
      
def check_author_name(author):
    author_names = author.strip().split(" ")
    author_names = [x for x in author_names if x != " "]
    
    if len(author_names) == 1:
        pass
    elif len(author_names) == 2:
        if len(author_names[1]) == 1:
            if author_names[0][-1]==",":
                author_names = [author_names[1].upper()] + [author_names[0][:-1]]
            else:
                author_names = [author_names[1].upper()] + [author_names[0]]
        elif len(author_names[1])==2:
            if author_names[1][1] in [".",",",";",":"]:
                if author_names[0][-1]==",":
                    author_names = [author_names[1].upper()] + [author_names[0][:-1]]
                else:
                    author_names = [author_names[1].upper()] + [author_names[0]]
            elif ((author_names[1].lower()[0] not in ["a","e","i","o","u"]) and 
                  (author_names[1].lower()[1] not in ["a","e","i","o","u"]) and 
                  (re.match(r'[a-zA-Z]{2}', author_names[1])) and 
                  (author_names[1].lower() not in ['ng','ty'])):
                author_names = [author_names[1].upper()] + [author_names[0]]
        elif len(author_names[1]) >=3:
            if author_names[0][-1] == ",":
                author_names = [author_names[1]] + [author_names[0][:-1]]
        else:
            pass
    elif len(author_names) == 3:
        pass
    elif len(author_names) >= 4:
        if ("(" in author_names) and (")" in author_names):
            temp_names = author_names[author_names.index("(")+1:author_names.index(")")]
            if len(temp_names) > 1:
                author_names = temp_names
            else:
                author_names = author_names[:author_names.index("(")]
    else:
        pass
    
    return " ".join(author_names)

def last_name_check(author_name):
    author_last_name_len = 0
    names = author_name.split(" ")
    if len(names) == 2:
        author_last_name_len = len(names[1])
        
    return author_last_name_len

def transform_coauthors(coauthors):
    if isinstance(coauthors, list):
        final_coauthors = []
        for coauthor in coauthors:
            final_coauthors.append(check_author_name(coauthor))
    else:
        final_coauthors = []
    return final_coauthors
  
def remove_current_author(author, coauthors):
    return [x for x in coauthors if x!=author]

def get_block_name_id(name):
    person = HumanName(name)
    last_name = person.last
    first_name = person.first
    if (len(first_name) < 1) & (len(last_name) < 1):
        return name.lower()
    elif len(first_name) < 1:
        return last_name
    elif len(last_name) < 1:
        return name.lower()
    else:
        initials = "%s_%s" % (first_name[0], last_name)
        return initials.lower()
            

turn_string_into_list_udf = F.udf(turn_string_into_list, ArrayType(StringType()))
add_W_to_works_and_authors_udf = F.udf(add_W_to_works_and_authors, StringType())
check_author_name_udf = F.udf(check_author_name, StringType())
transform_coauthors_udf = F.udf(transform_coauthors,  ArrayType(StringType()))
last_name_check_udf = F.udf(last_name_check, IntegerType())
remove_current_author_udf = F.udf(remove_current_author,  ArrayType(StringType()))
get_block_name_id_udf = F.udf(get_block_name_id, StringType())
add_W_to_references_udf = F.udf(add_W_to_references, ArrayType(StringType()))

In [0]:
test_df = raw_input_data \
.withColumn("affiliation", turn_string_into_list_udf(F.col('affiliation'))) \
.withColumn("coauthors", turn_string_into_list_udf(F.col('coauthors'))) \
.withColumn("references", turn_string_into_list_udf(F.col('references'))) \
.withColumn("UID", add_W_to_works_and_authors_udf(F.col('uid'))) \
.withColumn("paper_author_id", add_W_to_works_and_authors_udf(F.col('paper_author_id'))) \
.withColumn("author", check_author_name_udf(F.col('author'))) \
.withColumn("coauthors", transform_coauthors_udf(F.col('coauthors'))) \
.withColumn("block_id", get_block_name_id_udf('author'))

In [0]:
raw_input_data \
.withColumn("affiliation", turn_string_into_list_udf(F.col('affiliation'))) \
.withColumn("coauthors", turn_string_into_list_udf(F.col('coauthors'))) \
.withColumn("references", turn_string_into_list_udf(F.col('references'))) \
.withColumn("references", add_W_to_references_udf(F.col('references'))) \
.withColumn("UID", add_W_to_works_and_authors_udf(F.col('uid'))) \
.withColumn("paper_author_id", add_W_to_works_and_authors_udf(F.col('paper_author_id'))) \
.withColumn("author", check_author_name_udf(F.col('author'))) \
.withColumn("coauthors", transform_coauthors_udf(F.col('coauthors'))) \
.withColumn("coauthors", remove_current_author_udf(F.col('author'), F.col('coauthors'))) \
.withColumn("block_id", get_block_name_id_udf('author')) \
.write.mode('overwrite').parquet(f"{iteration_save_path}all_data_blocked")

#### Getting block IDs into correct file buckets

In [0]:
import random

In [0]:
author_block_ids = spark.read.parquet(f"{iteration_save_path}all_data_blocked").select('block_id').distinct()

In [0]:
author_block_ids.cache().count()

Out[13]: 25955385

In [0]:
def get_random_int(block_id):
    return random.randint(0,500)

get_random_int_udf = F.udf(get_random_int, IntegerType())

In [0]:
author_block_ids \
.withColumn("random_partition_number", get_random_int_udf('block_id')) \
.write.mode('overwrite').parquet(f"{iteration_save_path}data_block_partitions")

#### Getting data partitioned

In [0]:
all_data = spark.read.parquet(f"{iteration_save_path}all_data_blocked")
author_blocks = spark.read.parquet(f"{iteration_save_path}data_block_partitions")

In [0]:
partition_data = all_data.join(author_blocks, on='block_id')
partition_data.cache().count()

Out[10]: 604870974

In [0]:
partition_data \
    .repartition(1) \
    .write \
    .partitionBy("random_partition_number") \
    .mode("overwrite") \
    .json(f"{iteration_save_path}all_data_blocked_and_partitioned")

### Saving Affliation to orcid mapping

In [0]:
affiliations.filter(F.col('original_orcid')!='').show(5)

+----------+----------+--------------------+----------------------+--------------------+-------------------+
|  paper_id| author_id|     original_author|author_sequence_number|original_affiliation|     original_orcid|
+----------+----------+--------------------+----------------------+--------------------+-------------------+
|2909119052|2038892679|   Karen H. Rosenlof|                     4|                    |0000-0002-0903-8270|
|2909119052|2165641823|       Amy H. Butler|                     1|                    |0000-0002-3632-0925|
|2909119052|2342711310| Jeremiah P. Sjoberg|                     2|                    |0000-0002-7606-0566|
|2910448828| 672377543|Eric M. Leibenspe...|                    12|                    |0000-0002-1906-2688|
|2910448828|2010820540|    William H. Brune|                     4|                    |0000-0002-1609-4051|
+----------+----------+--------------------+----------------------+--------------------+-------------------+
only showing top 5 

In [0]:
affiliations.filter(F.col('original_orcid')!='') \
.select('paper_id','author_id','author_sequence_number','original_orcid') \
.coalesce(1).write.mode('overwrite').parquet(f"{iteration_save_path}paperid_authorid_orcid_mapping")

### Data for Whitepaper

In [0]:
W4299872130

In [0]:
raw_input_data = spark.read.parquet(f"{iteration_save_path}raw_author_disam_input_data").filter(F.col('author')!='')

In [0]:
raw_input_data.cache().count()

Out[5]: 604871070

In [0]:
raw_input_data.printSchema()

root
 |-- uid: long (nullable = true)
 |-- title: string (nullable = true)
 |-- pub_year: integer (nullable = true)
 |-- journal: string (nullable = true)
 |-- mag_author_id: long (nullable = true)
 |-- author: string (nullable = true)
 |-- paper_author_id: string (nullable = true)
 |-- affiliation: string (nullable = true)
 |-- seq_no: integer (nullable = true)
 |-- coauthors: string (nullable = true)
 |-- references: string (nullable = true)



In [0]:
raw_input_data.filter(F.col('uid')==2227647665).show(25)

+----------+--------------------+--------+-------+-------------+----------------+---------------+-----------+------+--------------------+----------+
|       uid|               title|pub_year|journal|mag_author_id|          author|paper_author_id|affiliation|seq_no|           coauthors|references|
+----------+--------------------+--------+-------+-------------+----------------+---------------+-----------+------+--------------------+----------+
|2227647665|Foreward to Meani...|    2015|   null|   2208157607|     Priem Jason|   2227647665_3|           |     3|Priem Jason::::te...|      null|
|2227647665|Foreward to Meani...|    2015|   null|   2215492700| Piwowar Heather|   2227647665_2|           |     2|Priem Jason::::te...|      null|
|2227647665|Foreward to Meani...|    2015|   null|   2197711048|team Impactstory|   2227647665_1|           |     1|Priem Jason::::te...|      null|
+----------+--------------------+--------+-------+-------------+----------------+---------------+---------

In [0]:
raw_input_data.groupby('mag_author_id').count().count()

Out[8]: 252973688

In [0]:
all_data_blocked = spark.read.parquet(f"{iteration_save_path}all_data_blocked")
all_data_blocked.cache().count()

Out[7]: 604871070

In [0]:
all_data_blocked.filter(F.col('block_id')=='p_jason').filter(F.col('author')=='Priem Jason').show(100)

+-----------+--------------------+--------+-------+-------------+-----------+---------------+-----------+------+--------------------+--------------------+--------+
|        UID|               title|pub_year|journal|mag_author_id|     author|paper_author_id|affiliation|seq_no|           coauthors|          references|block_id|
+-----------+--------------------+--------+-------+-------------+-----------+---------------+-----------+------+--------------------+--------------------+--------+
|W2227647665|Foreward to Meani...|    2015|   null|   2208157607|Priem Jason|  W2227647665_3|         []|     3|[team Impactstory...|                  []| p_jason|
|W2244341329|Toward a comprehe...|    2013|   null|   2208157607|Priem Jason|  W2244341329_2|         []|     2|[Piwowar Heather,...|[W2121727515, W25...| p_jason|
|  W87230806|Prevalence and us...|    2012|   null|   2208157607|Priem Jason|    W87230806_1|         []|     1|[Costello Kaitlin...|                  []| p_jason|
+-----------+---

In [0]:
all_data_blocked.dropDuplicates(subset=['UID']).count()

Out[18]: 221765881

In [0]:
116729526/221765881

Out[23]: 0.526363773695197

In [0]:
all_data_blocked.dropDuplicates(subset=['UID']) \
.select('title','journal','pub_year','mag_author_id','author','seq_no') \
.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in ['title','journal','pub_year','mag_author_id','author','seq_no']]).show()

+-----+-------+--------+-------------+------+------+
|title|journal|pub_year|mag_author_id|author|seq_no|
+-----+-------+--------+-------------+------+------+
|    1|      0|       0|            0|     0|     0|
+-----+-------+--------+-------------+------+------+



In [0]:
all_data_blocked.dropDuplicates(subset=['UID']) \
.withColumn('journal_check', F.when(F.col('journal')!="", 1).otherwise(0)) \
.select(F.sum(F.col('journal_check'))).show()

+------------------+
|sum(journal_check)|
+------------------+
|         116729526|
+------------------+



In [0]:
all_data_blocked \
.withColumn("affiliation_len", F.size(F.col('affiliation'))) \
.withColumn("coauthors_len", F.size(F.col('coauthors'))) \
.withColumn("references_len", F.size(F.col('references'))) \
.select([F.count(F.when(F.col(c)>0, 1)).alias(c) for c in ['affiliation_len','coauthors_len','references_len']]) \
.show()

+---------------+-------------+--------------+
|affiliation_len|coauthors_len|references_len|
+---------------+-------------+--------------+
|      604871070|    502486792|     253079634|
+---------------+-------------+--------------+



In [0]:
all_data_blocked \
.withColumn("affiliation_first", F.col('affiliation').getItem(0)) \
.withColumn('aff_check', F.when(F.col('affiliation_first')=="", 1).otherwise(0)) \
.select(F.sum(F.col('aff_check'))).show()

+--------------+
|sum(aff_check)|
+--------------+
|     320935815|
+--------------+



In [0]:
#missing
320935815/604871070

Out[29]: 0.5305854932027084

In [0]:
# complete
502486792/604871070

Out[24]: 0.8307337165257382

In [0]:
# complete
253079634/604871070

Out[25]: 0.41840260933623424